In [ ]:
import mysql.connector
import pandas as pd
import os

In [7]:
# Configura tus credenciales
DB_HOST = "localhost"      # O "127.0.0.1", o el nombre del servicio (por ejemplo, "mysql" en Docker)
DB_USER = "airflow"
DB_PASSWORD = "airflow"
DB_NAME = "airflow"

csv_file = "data/penguins_size.csv"  # Ruta al archivo CSV

In [8]:
# ---------------------------------------------------------
# Paso 2: Función para verificar archivo CSV
# ---------------------------------------------------------
def file_validator(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"El archivo {filepath} no existe.")
    print(f"El archivo {filepath} existe. OK!")

In [9]:
# ---------------------------------------------------------
# Paso 3: Función para crear (o reiniciar) la tabla 'penguins'
# ---------------------------------------------------------
def drop_and_create_table():
    try:
        conn = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # Eliminar la tabla si existe
        cursor.execute("DROP TABLE IF EXISTS penguins;")

        # Crear la tabla
        cursor.execute('''
            CREATE TABLE penguins (
                id INT AUTO_INCREMENT PRIMARY KEY,
                species VARCHAR(50),
                island VARCHAR(50),
                culmen_length_mm FLOAT,
                culmen_depth_mm FLOAT,
                flipper_length_mm FLOAT,
                body_mass_g FLOAT,
                sex VARCHAR(10)
            );
        ''')
        conn.commit()
        print("Tabla 'penguins' creada (o reiniciada) correctamente.")
    except Exception as e:
        print(f"Error al crear/reiniciar la tabla: {e}")
    finally:
        cursor.close()
        conn.close()

In [10]:
# ---------------------------------------------------------
# Paso 4: Función para cargar datos en la tabla 'penguins'
# ---------------------------------------------------------
def load_data_to_mysql(filepath):
    try:
        # Leer el CSV con pandas
        df = pd.read_csv(filepath)
        # Eliminamos filas con nulos
        df = df.dropna()

        conn = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # Inserción fila por fila
        for _, row in df.iterrows():
            cursor.execute(
                """
                INSERT INTO penguins 
                (species, island, culmen_length_mm, culmen_depth_mm, flipper_length_mm, body_mass_g, sex)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
                """,
                (
                    row['species'],
                    row['island'],
                    row['culmen_length_mm'],
                    row['culmen_depth_mm'],
                    row['flipper_length_mm'],
                    row['body_mass_g'],
                    row['sex']
                )
            )
        conn.commit()
        print("Datos insertados correctamente en la tabla 'penguins'.")
    except Exception as e:
        print(f"Error al cargar los datos: {e}")
    finally:
        cursor.close()
        conn.close()

In [11]:
# ---------------------------------------------------------
# Paso 5: Ejecutar todo en orden
# ---------------------------------------------------------

# 1. Validar archivo
file_validator(csv_file)

# 2. Crear (o reiniciar) la tabla
drop_and_create_table()

# 3. Cargar datos
load_data_to_mysql(csv_file)

print("Proceso completado.")

El archivo data/penguins_size.csv existe. OK!
Tabla 'penguins' creada (o reiniciada) correctamente.
Datos insertados correctamente en la tabla 'penguins'.
Proceso completado.
